In [1]:
import numpy as np
import datetime
import cv2
import torch
from absl import app, flags, logging
from absl.flags import FLAGS
from deep_sort_realtime.deepsort_tracker import DeepSort
from super_gradients.training import models
from super_gradients.common.object_names import Models
import threading
from queue import Queue
import sys

In [2]:
# 定义了一些命令行参数，使得你可以在运行脚本时通过命令行来传递特定的值
flags.DEFINE_string('f', 'value', 'The explanation of this parameter')
flags.DEFINE_string('model', 'yolo_nas_l', 'yolo_nas_l or yolo_nas_m or yolo_nas_s')
flags.DEFINE_string('video', "test.mp4", 'path to input video or set to 0 for webcam')
flags.DEFINE_string('output', "output.mp4", 'path to output video')
flags.DEFINE_float('conf', 0.50, 'confidence threshhold')

In [3]:

# 主函数，从命令行参数解析参数并执行处理逻辑
def main(_argv):
    
    video_cap = cv2.VideoCapture(FLAGS.video)
    #获得帧的宽度
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    #获得帧的高度
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    #获取视频的帧率（帧每秒）
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # 初始化视频写入对象
    #创建一个FourCC（四字符代码）对象，用于指定视频编码格式
    #FourCC 'MP4V' 表示使用 MPEG-4 Part 2 编码，通常用于生成MP4格式的视频文件。FourCC是一种用于标识视频编码格式的标准。
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    #创建一个名为 writer 的视频写入对象它接受四个参数：
    #FLAGS.output: 这是之前定义的命令行参数，表示输出视频的路径和文件名。
    #fourcc: 这是上一行创建的FourCC对象，指定了视频编码格式。
    #fps: 视频的帧率，用于指定写入的视频的帧率。
    #(frame_width, frame_height): 这是视频帧的宽度和高度，用于指定写入的视频的分辨率。
    writer = cv2.VideoWriter(FLAGS.output, fourcc, fps, (frame_width, frame_height))

    # 初始化 DeepSort 跟踪器
    #max_age 是一个参数，用于指定跟踪目标的最大帧数。如果目标在超过这个帧数之后仍然没有被检测到，它将被视为已失去跟踪，从跟踪器中移除。
    tracker = DeepSort(max_age=50)

    # 检查是否可用 GPU，否则使用 CPU
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

    # 加载 YOLO 模型
    model = models.get(FLAGS.model, pretrained_weights="coco").to(device)

    # 加载 YOLO 模型所训练的 COCO 类别标签
    classes_path = "coco.names"
    #f.read() 读取了文件的内容，
    #.strip() 方法去除了每行末尾的空白字符（如换行符），
    #然后 .split("\n") 方法将文件内容按行分割成一个列表
    with open(classes_path, "r") as f:
        class_names = f.read().strip().split("\n")

    # 创建一个随机颜色列表来表示每个类别
    np.random.seed(42)  # 设置随机数种子，以确保每次运行生成的随机数相同
    #生成随机数组，行数为目标类别数即为coco.names的行数，列数为3表示有三个颜色通道（红绿蓝），数值为0-255，每一行都是该类别颜色的RGB值
    colors = np.random.randint(0, 255, size=(len(class_names), 3))  
    # 在主函数内定义鼠标事件处理函数
    selected_object_id = -1
    pixels_per_meter = 1  # 像素与米的映射关系，示例值
    time_interval = 0
    # 初始化速度变量
    velocity = 0.0
    # 在主函数之前定义一个变量来保存截图状态和帧列表
    capture_screenshot = False
    frame_list = []
    def on_mouse_click(event, x, y, flags, param):
        nonlocal selected_object_id
        if event == cv2.EVENT_LBUTTONDOWN:  # 当鼠标左键点击时
            for idx, track in enumerate(tracks):
                ltrb = track.to_tlbr()  # 左上角和右下角坐标
                x1, y1, x2, y2 = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
                if x1 <= x <= x2 and y1 <= y <= y2:
                    selected_object_id = idx
                    print(f"Selected object with ID: {selected_object_id}")
                    break
        elif event == cv2.EVENT_RBUTTONDOWN:  # 检测右键点击事件
            nonlocal capture_screenshot
            capture_screenshot = True
   


    # 初始化视频写入对象
    fourcc_1 = cv2.VideoWriter_fourcc(*'mp4v')
    output_filename = 'single.mp4'
    output_fps = 8  # 设置输出视频的帧率
    output_size = (frame_width, frame_height)
    writer_1 = cv2.VideoWriter(output_filename, fourcc_1, output_fps, output_size)
    
    while True:
        # 调用当前时间语句记录开始时间以计算 FPS
        start = datetime.datetime.now()
        
        # 从视频捕获中读取一帧，设定ret布尔值来判断是否结束循环
        ret, frame = video_cap.read()
        # 在这部分加入鼠标回应函数，一旦点击，ret值设置为0，跳出循环，执行接下来的循环，实现点击目标框专门跟踪某人的功能
       
        # 如果没有帧，说明已经到达视频末尾或鼠标选择
        if not ret:
            print("End of the video file...")
            break

        # 对帧运行 YOLO 模型进行目标检测
        detect = next(iter(model.predict(frame, iou=0.5, conf=FLAGS.conf)))

        # 从检测结果中提取边界框坐标、置信度分数和类别标签
        bboxes_xyxy = torch.from_numpy(detect.prediction.bboxes_xyxy).tolist()
        confidence = torch.from_numpy(detect.prediction.confidence).tolist()
        labels = torch.from_numpy(detect.prediction.labels).tolist()
        
        # 将边界框坐标和置信度分数合并为一个列表
        concate = [sublist + [element] for sublist, element in zip(bboxes_xyxy, confidence)]
        
        # 将合并的列表与类别标签合并为最终的预测列表
        final_prediction = [sublist + [element] for sublist, element in zip(concate, labels)]

        # 初始化边界框和置信度列表
        results = []
        
        # 遍历检测结果
        for data in final_prediction:
            confidence = data[4]  # 提取与检测相关的置信度

            # 过滤掉置信度小于阈值的弱检测
            if float(confidence) < FLAGS.conf:
                continue

            # 如果置信度大于阈值，将边界框绘制在帧上
            xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
            class_id = int(data[5])
            
            # 将边界框（x、y、w、h）、置信度和类别 ID 添加到结果列表
            results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

        # 使用新的检测结果更新跟踪器
        tracks = tracker.update_tracks(results, frame=frame)
        for idx, track in enumerate(tracks):
            if selected_object_id != -1 and idx != selected_object_id:
                continue
            if selected_object_id != -1 and idx == selected_object_id:
                # 在这里添加处理特定对象的代码
                ltrb = track.to_ltrb()  # 获取跟踪信息
                class_id = track.get_det_class()
                

                x1, y1, x2, y2 = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
                captured_image = frame[y1:y2, x1:x2]  # 截取图像
                cv2.imwrite("captured_image.jpg", captured_image)
                # 删除选定对象的原有边界框和文本
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)  # 用白色绘制边界框
                cv2.putText(frame, "", (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)  # 清空文本
        # 目标对象框设置为红色
                B, G, R = 0,0,255

        # 创建显示在帧上的文本
                class_name = class_names[class_id]
                text = f"{track_id} - {class_name}(Selected) "

        # 在帧上绘制边界框和文本
                cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
                cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(text) * 12, y1), (B, G, R), -1)
                cv2.putText(frame, text, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
               
        # 遍历跟踪结果
        #for track in tracks:
        for idx, track in enumerate(tracks):
                # 如果跟踪未确认，忽略它
            if not track.is_confirmed():
                continue
            if selected_object_id != -1 and idx == selected_object_id:
                continue

            # 获取跟踪 ID 和边界框
            track_id = track.track_id
            ltrb = track.to_ltrb()
            class_id = track.get_det_class()
            # 如果有之前的帧，计算速度
            if hasattr(track, 'has_previous_frame') and track.has_previous_frame:
    # 在这里添加处理速度计算的代码

                prev_ltrb = track.to_ltrb(previous=True)

            # 计算像素变化
                pixel_change_x = ltrb[0] - prev_ltrb[0]
                pixel_change_y = ltrb[1] - prev_ltrb[1]

            # 将像素变化映射到实际距离变化
                distance_change_x = pixel_change_x / pixels_per_meter
                distance_change_y = pixel_change_y / pixels_per_meter

            # 计算速度，返回上一个帧和当前帧之间的时间差，以秒为单位
                time_interval = track.get_time_interval()
                if time_interval > 0:
                    speed_x = distance_change_x / time_interval
                    speed_y = distance_change_y / time_interval
                    velocity= np.sqrt(speed_x**2+speed_y**2)
                else:
                    velocity = 0.0
            x1, y1, x2, y2 = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
            
            # 获取类别的颜色
            color = colors[class_id]
            B, G, R = int(color[0]), int(color[1]), int(color[2])
            
            # 创建显示在帧上的文本
            class_name = class_names[class_id]
            text = f"{track_id} - {class_name}-{velocity} "

            # 在帧上绘制边界框和文本
            cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
            cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(text) * 12, y1), (B, G, R), -1)
            cv2.putText(frame, text, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # 在循环内判断是否需要截图
        if capture_screenshot:
        # 在选定对象框内截取图像
            if selected_object_id != -1:
                ltrb = tracks[selected_object_id].to_ltrb()
                x1, y1, x2, y2 = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
                captured_image_1 = frame[y1:y2, x1:x2]  # 截取图像
                frame_list.append(captured_image_1)  # 将图像添加到帧列表
        # 将每一帧的图像写入视频文件
        
       # 在循环之前，对捕获的帧进行调整大小
        output_frame_list = []
        for frame_img in frame_list:
            resized_frame = cv2.resize(frame_img, output_size)
            output_frame_list.append(resized_frame)
            

# 将调整大小后的帧写入视频
        for resized_frame in output_frame_list:
            writer_1.write(resized_frame)
        output_frame_list.clear()
        frame_list.clear()

        # 记录结束时间以计算 FPS
        end = datetime.datetime.now()
        
        # 显示处理 1 帧所需时间
        print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} milliseconds")
        
        # 计算并绘制 FPS
        fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
        cv2.putText(frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

        
        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        # 调整窗口大小以适应视频分辨率
        cv2.resizeWindow("Frame", frame_width, frame_height)
        # 显示帧
        cv2.imshow("Frame", frame)
        
        # 将帧写入输出视频文件
        writer.write(frame)
        cv2.setMouseCallback("Frame", on_mouse_click)  # 设置鼠标事件回调函数
       


        # 检查是否按下 'q' 键来退出循环
        if cv2.waitKey(1) == ord("q"):
            break

    # 释放视频捕获和视频写入对象
    video_cap.release()
    writer.release()
    writer_1.release()

    # 关闭所有窗口
    cv2.destroyAllWindows()

In [4]:
if __name__ == '__main__':
    try:
        #调用 absl 库的一个部分，用于运行主函数 main，并且可以处理命令行参数
        app.run(main)

    except SystemExit:
        pass